In [1]:
import json
import pandas as pd
import numpy as np
import requests
import time
import datetime as dt
import gc

In [2]:
#do I want to update from a preexisting csv?
update = False
#Which subreddit?
subreddit = 'Bitcoin'

In [3]:
def epoch_to_date(epoch_time):
    return dt.datetime.fromtimestamp(epoch_time).strftime('%Y-%m-%d %H:%M:%S')

def date_to_epoch(df_date):
    pattern = '%Y-%m-%d %H:%M:%S'
    return int(time.mktime(time.strptime(df_date, pattern)))

def rq(moving_date):
    #generates a request before 'moving_date'
    #if update. requests before 'moving' and after 'last_date'
    if update==False:
        r = requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit={}&size=500&is_video=False&before={}".format(subreddit,moving_date))
    else:
        r =  requests.get("https://api.pushshift.io/reddit/search/submission/"+
             "?subreddit={}&size=500&is_video=False&before={}&after={}".format(subreddit,moving_date,last_date))
    return r

In [4]:
if update==True:
    #imports previous csv
    df_0 = pd.read_csv('./Data/Reddit_{}_PushShift.csv'.format(subreddit),index_col=0)    
    #finds 24h before most recent value
    last_date = date_to_epoch(df_0.date.max())-(86400 * 2)#how many days ago do we want to overwrite?
    #reassigns df to df before 24h before last update
    #as to assume there may have been changes to 24h posts
    df_0 = df_0[df_0['date'] > epoch_to_date(last_date)]

In [8]:
moving_date = round(time.time())

In [9]:
#Starts as 'now', but moves with iteration
init = True
#df = pd.DataFrame()

while (init or len(r.json()['data'])>0):
    init = False
    n_ls = []
    
    print('Searching Dates before {}'.format(epoch_to_date(moving_date)))
    t0=time.time()

    r = rq(moving_date)
    #pushshifts limit is 200r/min
    # @ 1.5 sec, we are under 100r/min
    time.sleep(1.5)

    for sub in r.json()['data']:
        n_dic = {}
        n_dic['date'] = epoch_to_date(sub['created_utc'])
        n_dic['author'] = sub['author']
        n_dic['comments'] = sub['num_comments']
        n_dic['score'] = sub['score']
        n_dic['title'] = sub['title']
        n_dic['url'] = sub['url']
        n_ls.append(n_dic)
    df = df.append(pd.DataFrame(n_ls))

    #moving search window
    moving_date = date_to_epoch(df.date.min())
    
    t1=time.time()
    print('Search took {} seconds\n'.format(round(t1-t0)))

Searching Dates before 2017-05-03 02:30:55


NameError: name 'n_dic' is not defined

In [13]:
if update:
    df = df.append(df_0).sort_values('date',ascending=True)

In [14]:
#df.to_csv('./Data/Reddit_{}_PushShift.csv'.format(subreddit))
df.to_csv('./Data/Reddit_PushShift.csv')